Word similarity in python
https://stackoverflow.com/questions/17388213/find-the-similarity-percent-between-two-strings

In [121]:
import pandas as pd
import json
import operator
import re
from difflib import SequenceMatcher
import distance
import jellyfish
from collections import Counter
import os

## Eksisozluk Derlemin Okunması
- Satırlar Json formatından liste formatına çevriliyor

In [122]:
with open("icerik.jl") as f:
    lines = f.readlines()
lines_js = [json.loads(line) for line in lines]

## Listenin Pandas Dataframe'e Çevrilmesi
- URL, Page Number, ID, Author gibi alanlarda metin işlemleri

In [123]:
lines_df = pd.DataFrame(lines_js)
lines_df["url"] = lines_df.url.apply(lambda x:x[23:])
#lines_df["topic"] = lines_df.url.apply(lambda x:x.split("?")[0])
lines_df["page_number"] = lines_df.url.apply(lambda x:x.split("?")[1].split("&")[1].split("=")[1])
del lines_df["url"]
lines_df["id_s"] = lines_df.id.apply(lambda x:str(x)[2:-2])
del lines_df["id"]
lines_df["author_s"] = lines_df.author.apply(lambda x:str(x)[2:-2])
del lines_df["author"]

## Nihai DataFrame'in görüntüsü

In [124]:
lines_df.head()

,content,header,page_number,id_s,author_s
0,\r\n aşırı uzun yazayım ki derdim büyük gör...,9 şubat 2018 apple watch rezaleti,3,74106621,import os
1,\r\n garmin ve apple arasında kalıp apple'ı...,9 şubat 2018 apple watch rezaleti,3,74106708,whokares
2,\r\n hergün en az iki tane (rezalet olmayan...,9 şubat 2018 apple watch rezaleti,3,74106719,yuzuklerinprensesi
3,\r\n (bkz: birisine sen malsın diyememek )...,9 şubat 2018 apple watch rezaleti,3,74106730,bagirmayan taraftar
4,\r\n --- spoiler --- elbette bu durumunda...,9 şubat 2018 apple watch rezaleti,3,74106744,metamorfik dalak


## Girdilerin Kelimelere Ayrıştırılması
- Yeni satır karakterleri temizleniyor

In [126]:
lines_df.count()

content        1630
header         1630
page_number    1630
id_s           1630
author_s       1630
dtype: int64

In [127]:
lines_df.content.head()

0    \r\n    aşırı uzun yazayım ki derdim büyük gör...
1    \r\n    garmin ve apple arasında kalıp apple'ı...
2    \r\n    hergün en az iki tane (rezalet olmayan...
3    \r\n    (bkz:  birisine sen malsın diyememek )...
4    \r\n    ---  spoiler  --- elbette bu durumunda...
Name: content, dtype: object

In [128]:
contents = lines_df.content.apply(lambda x: x.replace("\r\n",""))

## Gereksiz Kelimeler

In [129]:
stoplist = open("turkce-stop-words",encoding="utf8").readlines()
stoplist = [x.replace("\n","") for x in stoplist]

In [130]:
stoplist.append("bkz")
stoplist.append("amk")
stoplist.append("edit")
stoplist.append("lan")


## Referans Kelime Sözlüğünün Oluşturulması
- Kelimenin uzunluğu yetersiz ise temizlenir
- Nümerik ve özel karakterler temizlenir
- Stoplist içerisinde geçiyorsa temizlenir
- Bu kurallardan geçmiş ise, sözlük listesinde dökümanlarda görülme frekansı ile birlikte eklenir

In [131]:
bag_of_words = dict()
for a in contents:
    t = a.lower().split(" ")
    t = [re.sub('\W+', '', x) for x in t]
    t = [re.sub('[0-9]+', '', x) for x in t]
    for x in t:
        if len(x) < 2:
            continue
        if len(x) > 30:
            continue
        if x[:4] == "http":
            continue
        if x in stoplist:
            continue
        if x in bag_of_words.keys():
            bag_of_words[x] += 1
        else:
            bag_of_words[x] = 1


## Dökümanlarda en sık geçen kelimeler

In [132]:
sorted(bag_of_words.items(),key=operator.itemgetter(1),reverse=True)[:100]

[('yok', 263),
 ('soy', 180),
 ('büyük', 180),
 ('bi', 141),
 ('aynı', 121),
 ('uygulama', 115),
 ('merak', 114),
 ('sistem', 110),
 ('insan', 107),
 ('türk', 104),
 ('edevlet', 103),
 ('baba', 96),
 ('hizmet', 95),
 ('saat', 94),
 ('sadece', 91),
 ('zaman', 88),
 ('güzel', 86),
 ('anne', 85),
 ('adam', 85),
 ('ilk', 83),
 ('nedir', 80),
 ('başka', 70),
 ('ulan', 70),
 ('iyi', 70),
 ('nüfus', 69),
 ('an', 68),
 ('gece', 68),
 ('fazla', 67),
 ('üzerinden', 65),
 ('son', 64),
 ('devam', 64),
 ('devlet', 64),
 ('olacak', 62),
 ('sorgulama', 62),
 ('saygı', 61),
 ('tek', 60),
 ('para', 60),
 ('gün', 59),
 ('sonunda', 57),
 ('geliyor', 56),
 ('ediyorum', 56),
 ('dedemin', 56),
 ('yeni', 54),
 ('tarafı', 54),
 ('babamın', 53),
 ('babasının', 53),
 ('yıl', 53),
 ('ermeni', 52),
 ('insanlar', 52),
 ('kürt', 50),
 ('geldi', 50),
 ('site', 50),
 ('tekrar', 49),
 ('tl', 48),
 ('altüst', 48),
 ('olabilir', 48),
 ('atatürk', 47),
 ('sanırım', 47),
 ('çıktı', 47),
 ('kişi', 46),
 ('millet', 45),
 ('

In [133]:
len_list = [len(x) for x in list(bag_of_words.keys())]
cnt = Counter()
for w in len_list:
    cnt[w] += 1
cnt

Counter({2: 98,
         3: 431,
         4: 860,
         5: 2007,
         6: 2398,
         7: 3023,
         8: 2949,
         9: 2680,
         10: 2112,
         11: 1567,
         12: 1046,
         13: 659,
         14: 381,
         15: 215,
         16: 141,
         17: 77,
         18: 38,
         19: 20,
         20: 20,
         21: 9,
         22: 5,
         23: 3,
         24: 2,
         25: 1,
         26: 2,
         27: 2,
         29: 1})

In [114]:
#[(x,len(x)) for x in list(bag_of_words.keys()) if len(x) == 2]

In [136]:
len(bag_of_words)

20747

In [235]:
cutoff = 1000
prob_cutoff = 0.75
i = 0
with open("word_sim.csv","w",encoding="utf8") as f:
    for word1 in bag_of_words:
        i += 1
        if i > cutoff:
            break
        j = 0
        for word2 in bag_of_words:
            j += 1
            if j > cutoff:
                break
            if word1 != word2:
                sim_score = SequenceMatcher(None,word1,word2).ratio()
                if sim_score > prob_cutoff:
                    f.write(word1 + "|" + word2 + "->" + str(sim_score) + "\r\n")


## Twitter Parsing

In [267]:
ss = [json.loads(open("twits/" + a).read()) for a in filenames]
ss[0]["user"]["location"]

'Bandung, West Java'

In [268]:
filenames = os.listdir("twits")
twit_list = []
for file in filenames:
    fullpath = "twits/" + file
    full_text = ""
    location = ""
    with open(fullpath,"r") as f:
        curr_tw = json.loads(f.read())
        if "text" not in curr_tw:
            continue
            
        text = curr_tw["text"]
        
        if "retweeted_status" in curr_tw.keys():
            if "extended_tweet" in curr_tw["retweeted_status"].keys():
                if "full_text" in curr_tw["retweeted_status"]["extended_tweet"].keys():
                    full_text = curr_tw["retweeted_status"]["extended_tweet"]["full_text"]
        if "user" in curr_tw.keys():
            if "location" in curr_tw["user"].keys():
                location = curr_tw["user"]["location"]
                    
        twit_list.append([text,full_text,location])
                    
        


In [300]:
delim = "||"
with open("tweets.csv","w",encoding="utf8") as f:
    for twit in twit_list:
        if len(str(twit[1])) > 2:
            #f.write(str(twit[1]) + delim + str(twit[2]) + "\r\n")
            content = str(twit[1])
        else:
            content = str(twit[0])
        content = content.replace("\r\n","")
        content = content.replace("\n","")
        f.write(content + delim + str(twit[2]) + "\r\n")
    

In [275]:
len(twit_list)

1786

In [33]:
with open("HUMIRSentimentDatasets.csv", encoding="utf8") as f:
    x = f.readlines()

In [36]:
x[:5]

['\ufeff80101;Hotel Review;asla gidilmeyecek bir otel hasta oldukotel tam anlamıyla bir fiyasko satın alırken ve web sitesinde gözünüze çarpan en büyük özellik otelin tüm alanlarının yenilenmiş olması ama bunun gerçekle alakası yok odalar en az  yıllık bir otel harabeliğinde yemekler ve özellikle kahvaltı tam bir hayal kırıklığı kahvaltıdaki yiyecekler asla yenmeyecek ve yedirilmeyecek kadar kötü bir tane lekesiz temiz bir tabak bardak veya çatal kaşık görmeniz olası bile değil içecek konusunda su değil zehirli su katılmış gibi gerçeği ile alakası olmayan içecekler ıce tea yada soğuk çay cinsi birşey otelde asla yok konsepte uygun değilmiş açıklama bu soğuk çay hangi konseptin ki acaba bu otele uymuyor garsonların hepsi kendi dalında bir kabadayı restaurant müdürü denen kişi inanılmaz yeteneksiz asla yeme içme kültürü yok gerçekten birsey isteyipte almanız mucize birde asıl bir mevzu varki anlatılmaz bu otelde can güvenliğiniz yok oda anahtarı kardeşimdeydi ben resepsiyona anahtar alma